In [1]:
import apache_beam as beam

In [9]:
p1 = beam.Pipeline()


In [10]:
cases = (
    p1
    | 'Reading cases data' >> beam.io.ReadFromText('data/datasets_527325_1205308_Case.csv')
    | 'Splitting records' >> beam.Map(lambda x: x.split(','))
    | 'Filtering Seoul' >> beam.Filter(lambda x: x[1]=='Seoul')
    | 'Creating Key value pairs' >> beam.Map(lambda x: (x[2],1))
    | 'Grouping & counting Keys' >> beam.CombinePerKey(sum)
    | 'Writing to file' >> beam.io.WriteToText('data/output_new_final')
)

In [11]:
p1.run()

In [13]:
with beam.Pipeline() as p:
    input_collection = (
        p
        | 'Reading cases data' >> beam.io.ReadFromText('data/datasets_527325_1205308_Case.csv')
        | 'Split rows' >> beam.Map(lambda x: x.split(','))
    )
    
    seoul_cases = (
        input_collection
        | 'Filtering Seoul' >> beam.Filter(lambda x: x[1]=='Seoul')
        | 'Creating key value pairs seoul' >> beam.Map(lambda x: ('Seoul ' + x[2], 1))
        | 'Grouping and counting keys seoul' >> beam.CombinePerKey(sum)
    )
    
    busan_cases = (
        input_collection
        | 'Filtering Busan' >> beam.Filter(lambda x: x[1]=='Busan')
        | 'Creating key value pairs busan' >> beam.Map(lambda x: ('Busan ' + x[2], 1))
        | 'Grouping and counting keys busan' >> beam.CombinePerKey(sum)
    )
    
    output = (
        (seoul_cases, busan_cases)
        | 'Combining both outputs' >> beam.Flatten()
        | 'Writing output' >> beam.io.WriteToText('data/output_combined_final')
    )

In [14]:
class SplitRow(beam.DoFn):
    def process(self, element):
        return [element.split(',')]
    
p = beam.Pipeline()

all_cases = (
    p
    | beam.io.ReadFromText('data/datasets_527325_1205308_Case.csv')
    | beam.ParDo(SplitRow())
    | beam.io.WriteToText('data/output_all_final')


)

In [15]:
p.run()